# b-33

In [ ]:
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scikeras.wrappers import KerasRegressor
from  sklearn.model_selection import GridSearchCV

In [ ]:
california_housing = fetch_california_housing()
california_housing

# Splitting data into train and test.

In [ ]:
X_train0, X_test, Y_train0, Y_test = train_test_split(california_housing["data"], california_housing["target"])

In [ ]:
X_train1, X_validation, Y_train1, Y_validation = train_test_split(X_train0, Y_train0)

In [ ]:
X_test.shape

In [ ]:
X_train0.shape

In [ ]:
X_train1.shape

In [ ]:
X_validation.shape

# Normalizing data
using "sklearn.preprocessing.StandardScaler" function for normalizing data

In [ ]:
sc = StandardScaler()
X_train_s = sc.fit_transform(X_train1)
X_validation_s = sc.fit_transform(X_validation)
X_test_s = sc.transform(X_test)

--------------------------------

In [ ]:
X_train0.shape[1:]

In [ ]:
X_train_s_1, X_train_s_2 = X_train_s[:, :6], X_train_s[:, :-4]
X_validation_s_1, X_validation_s_2 = X_validation_s[:, :6], X_validation_s[:, :-4]
X_test_s_1, X_test_s_2 = X_test_s[:, :6], X_test_s[:, :-4]

# Define Model + Compile with Hyperparameter Optimization

In [ ]:
def ann_model(number_of_hidden_layers=1,
              number_of_neurons=50,
              lr=0.01):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(shape=[8]))
    for hidden_layer in range(number_of_hidden_layers):
        model.add(keras.layers.Dense(number_of_neurons, activation="selu"))
    model.add(keras.layers.Dense(1))
    sgd = keras.optimizers.SGD(learning_rate=lr)
    model.compile(loss="mse", optimizer=sgd)
    return model

#### this is deprecated:
#### keras_sk_reg = keras.wrappers.scikit_learn.KerasRegressor(build_fn=ann_model)
#### instead using scikeras

In [ ]:
keras_sk_reg = KerasRegressor(model=ann_model)

In [ ]:
param_grid = {
    "model__number_of_hidden_layers": [1, 3, 5],
    "model__number_of_neurons": [50, 100, 150],
    "model__lr": [0.001, 0.01, 0.1]
}

In [ ]:
keras_sk_reg_gs = GridSearchCV(keras_sk_reg, param_grid)

In [ ]:
keras_sk_reg_gs.fit(X_train_s, Y_train1, epochs=1, verbose=0)

# Callbacks

In [ ]:
model_checkpoint_callback = keras.callbacks.ModelCheckpoint("model_cb_regression_housing.keras", save_best_only=True)
early_stopping_callback = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
tb_callback = keras.callbacks.TensorBoard("tb_log3")

# Compile Model

In [ ]:
model_f.compile(loss=["mse", "mse"],
                loss_weights=[0.8, 0.2],
                optimizer="sgd",
                metrics=[["mean_absolute_error"], ["mean_absolute_error"]])

# Fit Model

In [ ]:
model_f.fit((X_train_s_1, X_train_s_2), (Y_train1, Y_train1), epochs=5,
            validation_data=((X_validation_s_1, X_validation_s_2), (Y_validation, Y_validation)),
            callbacks=[tb_callback, model_checkpoint_callback, early_stopping_callback])

In [ ]:
model_f.save("housing_reg_model.keras")

In [ ]:
model_f_reg = keras.models.load_model("housing_reg_model.keras")

In [ ]:
model_f_reg.summary()